# [Omdena-NeedEnergy "Increasing Clean Energy Access in Africa" project](https://omdena.com/projects/clean-energy-ai/)
#### A.Montesino (Feb 10th, 2021)
### Task X - XYZ

References:
* [Document #1](#http://www.loremipzum.com/en/)
* [Document #2](#http://www.loremipzum.com/es/)
* other...


In [30]:
import numpy as np
from numpy import *
print( "NumPy version:", np.__version__ )
# pip show numpy

NumPy version: 1.19.1


In [31]:
import pandas as pd
from pandas import *
print( "Pandas version:", pandas.__version__ )

Pandas version: 1.1.0


In [26]:
# pip list
import numexpr
numexpr.print_versions()

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Numexpr version:   2.7.1
NumPy version:     1.19.1
Python version:    3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
Platform:          win32-AMD64-10.0.18362
CPU vendor:        GenuineIntel
CPU model:         Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz
CPU clock speed:   1992 MHz
VML available?     True
VML/MKL version:   Intel(R) Math Kernel Library Version 2020.0.1 Product Build 20200208 for Intel(R) 64 architecture applications
Number of threads used by default: 8 (out of 8 detected cores)
Maximum number of threads: 64
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [32]:
import seaborn as sns
import matplotlib.pyplot as plt

# Comment this if the data visualisations doesn't work on your side
%matplotlib inline

plt.style.use('bmh')

In [ ]:
import os

path ="C:/workspace/python"
#we shall store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))

#print all the file names
for name in filelist:
    print(name)

In [43]:
DataPath='C:\\Users\\alvar\\OneDrive\\Personal\\Repositorio\\Data Analytics\\Omdena\\Data'
DataPath='C:/Users/alvar/OneDrive/Personal/Repositorio/Data Analytics/Omdena/Data/NeedEnergy Meter Data/data for Mini Substation'
DataFile='Electricity usage - 2020-01-01 - 2021-02-05 - 5 minutes.xlsx'
from os import listdir
from os.path import isfile, join

onlyfiles = [f for f in listdir( DataPath ) if isfile( join( DataPath, f) ) ]
print( onlyfiles )


['Electricity usage - 2020-01-01 - 2021-02-05 - 5 minutes.xlsx', 'Electricity usage - 2020-01-01 - 2021-02-09 - daily.xlsx', 'Electricity usage - 2020-01-01 - 2021-02-09 - hourly.xlsx']


In [1]:
import os

path ='C:/Users/alvar/OneDrive/Personal/Repositorio/Data Analytics/Omdena/Data/NeedEnergy Meter Data/data for Mini Substation'
path ='C:/Users/alvar/OneDrive/Personal/Repositorio/Data Analytics/Omdena/Data'

#we shall store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
     for file in files:
        if( file.endswith( ".xlsx" ) ):
        #append the file name to the list
            filelist.append( os.path.join( root,file ) )



# Data Quality Check

# Quantitative tests

# Statistical test

# Visualization

In [71]:
Max_PRCT_NaNs = 0.3

#print all the file names
for filename in filelist:
    print( filename )
    df = pd.read_excel( filename )
    df.head( )
    df.info( )
    print( "Variable types " )    
    list( set( df.dtypes.tolist( ) ) )

    # pandas.DataFrame.dtypes    
    df_numeric = df.select_dtypes( include = [ 'int', 'int64', 'float', 'float64', 'complex' ] )
    df_boolean = df.select_dtypes( include = [ 'bool' ] )
    # df_bynary = df.select_dtypes( include = [ 'bytes', 'bytearray', 'memoryview' ] )
    df_time = df.select_dtypes( include =  [ 'datetime', 'datetime64' ] )
    df_categorical = df.select_dtypes( include = ['object' ] )
    
    for t in t
    df_num.head()

    # df.count() does not include NaN values
    df_aux = df[ [column for column in df if df[column].count() / len(df) >= Max_PRCT_NaNs]]
    df_aux.show()
#   del df_aux[ 'Id' ]
    print("List of dropped columns:", end=" ")
    for column in df.columns:
        if column not in df_aux.columns:
            print(c, end=", ")
        print('\n')
        df = df_aux
        


C:/Users/alvar/OneDrive/Personal/Repositorio/Data Analytics/Omdena/Data\NeedEnergy Meter Data\data for Mini Substation\Electricity usage - 2020-01-01 - 2021-02-05 - 5 minutes.xlsx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5044 entries, 0 to 5043
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              5044 non-null   object 
 1   Consumption [kW]  5044 non-null   float64
 2   Always on [kW]    5044 non-null   float64
dtypes: float64(2), object(1)
memory usage: 118.3+ KB
Variable types 


NameError: name 'df_num' is not defined